# Contents
1. [Setup](#Setup)
2. [Data Validation](#Data-Validation)
3. [EDA](#EDA)

# Setup

#### Import Libraries

In [ ]:
import requests
import pandas as pd
from datetime import datetime

#### Reading Data

In [ ]:
# Endpoint for CoinGecko's BTC daily market data
url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
params = {
    'vs_currency': 'usd',  # Currency in which price is returned
    'days': 'max',         # Get data for the maximum range available
    'interval': 'daily'    # Interval set to daily
}

# Requesting data from the API
response = requests.get(url, params=params)
data = response.json()

# Processing the response to create a DataFrame
dates = [datetime.fromtimestamp(item[0] / 1000).date() for item in data['prices']]
prices = [item[1] for item in data['prices']]

# Create DataFrame with date and price columns
df = pd.DataFrame({'Date': dates, 'Price': prices})

# Display the first few rows
print(df.head())

[Back to the top](#Contents)

# Data Validation

[Back to the top](#Contents)

# EDA

[Back to the top](#Contents)